In [24]:
import sys, os
import copy
import re

import pickle as pkl
import numpy as np
import pandas as pd

from sklearn.metrics import mean_squared_error
from scipy.stats import entropy
from scipy.spatial.distance import jensenshannon

# Load Data

In [ ]:

MIN_NUM_READS = 100

XCRISP_PREDICTIONS_F = "/Users/colm/repos/output/local/model_predictions/OurModel/model_1_v4_RS_1_{}.pkl"
XCRISP_KLD_PREDICTIONS_F = "/Users/colm/repos/output/local/model_predictions/OurModel/model_v4_kld_{}.pkl"
LINDEL_PREDICTIONS_F = "/Users/colm/repos/output/local/model_predictions/Lindel/predictions_{}x_{}.pkl"
FORECasT_PREDICTIONS_F = "/Users/colm/repos/output/local/model_predictions/FORECasT/predictions_{}x_{}.pkl"
INDELPHI_PREDICTIONS_F = "/Users/colm/repos/output/local/model_predictions/inDelphi/{}_predictions.pkl"
TEST_FILES = ["test", "0105-mESC-Lib1-Cas9-Tol2-BioRep2-techrep1", "052218-U2OS-+-LibA-postCas9-rep1_transfertest", "0226-PRLmESC-Lib1-Cas9_transfertest", "TREX_A_test", "HAP1_test"]
BASELINE_TEST_FILES = ["test", "0105-mESC-Lib1-Cas9-Tol2-BioRep2-techrep1"]
data = {}

for i, t in enumerate(TEST_FILES):
    data[t] = {}
    data[t]["X-CRISP-MSE"] = pkl.load(open(XCRISP_PREDICTIONS_F.format(t), 'rb'))
    data[t]["X-CRISP-KLD"] = pkl.load(open(XCRISP_KLD_PREDICTIONS_F.format(t), 'rb'))
    # data[t]["inDelphi"] = pkl.load(open(INDELPHI_PREDICTIONS_F.format(t), 'rb'))
    data[t]["Lindel"] = pkl.load(open(LINDEL_PREDICTIONS_F.format(MIN_NUM_READS, t), 'rb'))
    data[t]["FORECasT"] = pkl.load(open(FORECasT_PREDICTIONS_F.format(MIN_NUM_READS, t), 'rb'))

    # need to remove psuedocounts that were added to profiles in preparation for training
    for target_site in data[t]["FORECasT"].keys():
        data[t]["FORECasT"][target_site]["actual"] = np.array(data[t]["FORECasT"][target_site]["actual"]) - 0.5

models = list(data[t].keys())
print(models, "loaded")

['X-CRISP-MSE', 'X-CRISP-KLD', 'Lindel', 'FORECasT'] loaded


In [26]:
t = "052218-U2OS-+-LibA-postCas9-rep1_transfertest"
method = "X-CRISP-MSE"
target_site = "0_0_0_0_AAATATCTTTAACCTAAAAC"
print(t, method, target_site)

052218-U2OS-+-LibA-postCas9-rep1_transfertest X-CRISP-MSE 0_0_0_0_AAATATCTTTAACCTAAAAC


In [37]:
for t in TEST_FILES:
    for m in models:
        target_site = list(data[t][m].keys())[0]
        for k in data[t][m][target_site].keys():
            print(t, m, target_site, len(data[t][m][target_site][k]))

test X-CRISP-MSE Oligo_43170 434
test X-CRISP-MSE Oligo_43170 434
test X-CRISP-MSE Oligo_43170 434
test X-CRISP-MSE Oligo_43170 434
test X-CRISP-KLD Oligo_43170 434
test X-CRISP-KLD Oligo_43170 434
test X-CRISP-KLD Oligo_43170 434
test X-CRISP-KLD Oligo_43170 434
test Lindel Oligo_43170 557
test Lindel Oligo_43170 557
test Lindel Oligo_43170 557
test Lindel Oligo_43170 557
test FORECasT Oligo_43170 398
test FORECasT Oligo_43170 398
test FORECasT Oligo_43170 398
test FORECasT Oligo_43170 398
0105-mESC-Lib1-Cas9-Tol2-BioRep2-techrep1 X-CRISP-MSE 0_0_0_0_CTTTCACTTTATAGATTTAT 393
0105-mESC-Lib1-Cas9-Tol2-BioRep2-techrep1 X-CRISP-MSE 0_0_0_0_CTTTCACTTTATAGATTTAT 393
0105-mESC-Lib1-Cas9-Tol2-BioRep2-techrep1 X-CRISP-MSE 0_0_0_0_CTTTCACTTTATAGATTTAT 393
0105-mESC-Lib1-Cas9-Tol2-BioRep2-techrep1 X-CRISP-MSE 0_0_0_0_CTTTCACTTTATAGATTTAT 393
0105-mESC-Lib1-Cas9-Tol2-BioRep2-techrep1 X-CRISP-KLD 0_0_0_0_CTTTCACTTTATAGATTTAT 393
0105-mESC-Lib1-Cas9-Tol2-BioRep2-techrep1 X-CRISP-KLD 0_0_0_0_CTTTCAC

### Reformat files

In [27]:
for t in TEST_FILES:
    for e in data[t]:
        print(t, e, len(data[t][e].keys()))

# collect targets common to all experiments

from functools import reduce
common_oligos = {}


for t in TEST_FILES:
    ts = []
    ts.append(list(data[t]["Lindel"].keys()))
    ts.append(list(data[t]["FORECasT"].keys()))
    common_oligos[t] = reduce(np.intersect1d, ts)
    print(f"There are {len(common_oligos[t])} oligos common to {t}")

# reformat FORECasT indels
def FC_indel_to_our(fc):
    parts = fc.split("_")
    t = parts[0][0] # type I or D 
    l = int(parts[0][1:]) # size
    p = int(parts[1].split("R")[1]) # start
    if t == "D":
        return "{}+{}".format(p-l, l)
    else:
        # return "I{}".format(l)
        return fc

def inDelphi_to_our(ind):
    if ind[-1] in "ACGT":
        return "1+" + ind[-1]
    if ind.isnumeric():
        return "DL" + ind 
    return ind

for t in TEST_FILES:
    for o in common_oligos[t]:
        if "FORECasT" in data[t]:
            data[t]["FORECasT"][o]["indels"] = [FC_indel_to_our(i) for i in data[t]["FORECasT"][o]["indels"]]
        if "inDelphi" in data[t]:
            data[t]["inDelphi"][o]["indels"] = [inDelphi_to_our(i) for i in data[t]["inDelphi"][o]["indels"]]

# np.seterr(all='raise')
file_mapping = {
    "test": "FORECasT WT",
    "0105-mESC-Lib1-Cas9-Tol2-BioRep2-techrep1": "inDelphi WT",
    "WT": "LUMC WT",
    "0226-PRLmESC-Lib1-Cas9_transfertest": "inDelphi NHEJ-deficient",
    "052218-U2OS-+-LibA-postCas9-rep1_transfertest": "inDelphi USO2 WT",
    "HAP1_test": "FORECasT HAP1",
    "TREX_A_test": "FORECasT TREX",
    "2A_TREX_A_test": "2A_FORECasT TREX",
}

test X-CRISP-MSE 3954
test X-CRISP-KLD 3954
test Lindel 3939
test FORECasT 3939
0105-mESC-Lib1-Cas9-Tol2-BioRep2-techrep1 X-CRISP-MSE 1961
0105-mESC-Lib1-Cas9-Tol2-BioRep2-techrep1 X-CRISP-KLD 1961
0105-mESC-Lib1-Cas9-Tol2-BioRep2-techrep1 Lindel 1961
0105-mESC-Lib1-Cas9-Tol2-BioRep2-techrep1 FORECasT 1961
052218-U2OS-+-LibA-postCas9-rep1_transfertest X-CRISP-MSE 962
052218-U2OS-+-LibA-postCas9-rep1_transfertest X-CRISP-KLD 962
052218-U2OS-+-LibA-postCas9-rep1_transfertest Lindel 962
052218-U2OS-+-LibA-postCas9-rep1_transfertest FORECasT 962
0226-PRLmESC-Lib1-Cas9_transfertest X-CRISP-MSE 985
0226-PRLmESC-Lib1-Cas9_transfertest X-CRISP-KLD 985
0226-PRLmESC-Lib1-Cas9_transfertest Lindel 985
0226-PRLmESC-Lib1-Cas9_transfertest FORECasT 985
TREX_A_test X-CRISP-MSE 3355
TREX_A_test X-CRISP-KLD 3355
TREX_A_test Lindel 3355
TREX_A_test FORECasT 3949
HAP1_test X-CRISP-MSE 3950
HAP1_test X-CRISP-KLD 3950
HAP1_test Lindel 3837
HAP1_test FORECasT 3950
There are 3939 oligos common to test
There a

### Generate ins mappings to FORECasT output 

In [28]:
import re
sys.path.append("../modelling")
from src.data.data_loader import get_details_from_fasta

# test over a common set of indels per target site
def generate_1_and_2_bp_insertions():
    nucs = ["A", "C", "G", "T"]
    onebps = []
    twobps = []
    for n1 in nucs:
        onebps.append("1+" + n1)
        for n2 in nucs:
            twobps.append("2+" + n1 + n2)
    return onebps + twobps

common_insertions = generate_1_and_2_bp_insertions()

fasta_files = ["../../src/data/FORECasT/test.fasta", "../../src/data/inDelphi/LibA.fasta"]
guides = {}

for ff in fasta_files:
    guides.update(get_details_from_fasta(ff))

ins_mapping = {}
t = "test"
for t in TEST_FILES:
    ins_mapping[t] = {}
    for o in common_oligos[t]:
        g = guides[o]
        cutsite = g["PAM Index"] - 3
        ins_mapping[t][o] = {}
        FORECasT_insertions = [i for i in data[t]["FORECasT"][o]["indels"] if "I" in i]
        FORECasT_rep_insertions = [i for i in FORECasT_insertions if "C" in i]
        FORECasT_norep_insertions = [i for i in FORECasT_insertions if "C" not in i]
        if len(FORECasT_norep_insertions) not in [1, 2]: print("wtf")
        for i in FORECasT_rep_insertions:
            _, I, _, L, C, R = re.split("I|_|L|C|R", i)
            rep_nuc = g["TargetSequence"][cutsite + int(R) -int(I):cutsite + int(R)]
            ins_mapping[t][o][i] = "{}+{}".format(int(I), rep_nuc)
        for i in FORECasT_norep_insertions:
            if i[1] == "1":
                ins_mapping[t][o][i] = list(np.setdiff1d([c for c in common_insertions if "1" in c], list(ins_mapping[t][o].values()))) 
            if i[1] == "2":
                ins_mapping[t][o][i] = list(np.setdiff1d([c for c in common_insertions if "2" in c], [c for c in list(ins_mapping[t][o].values()) if "2" in c])) 
print("mapped output to FORECasT")
print(ins_mapping[t][o])

mapped output to FORECasT
{'I1_L-1C1R1': '1+T', 'I1_L-2C1R0': '1+A', 'I2_L-1C1R1': '2+AT', 'I2_L-1C2R2': '2+TC', 'I2_L-2C1R0': '2+GA', 'I2_L-2C2R1': '2+AT', 'I2_L-3C2R0': '2+GA', 'I1_L-1R0': ['1+C', '1+G'], 'I2_L-1R0': ['2+AA', '2+AC', '2+AG', '2+CA', '2+CC', '2+CG', '2+CT', '2+GC', '2+GG', '2+GT', '2+TA', '2+TG', '2+TT']}


In [29]:
rev_ins_mapping = {}
for t in ins_mapping:
    rev_ins_mapping[t] = {}
    for o in ins_mapping[t]:
        rev_ins_mapping[t][o] = {}
        for i in ins_mapping[t][o]:
            a = ins_mapping[t][o][i]
            if isinstance(a, list):
                for a2 in a:
                    rev_ins_mapping[t][o][a2] = i
            else:
               rev_ins_mapping[t][o][a] = i
print("Reversed mapping")

rev_ins_mapping[t][o]

Reversed mapping


{'1+T': 'I1_L-1C1R1',
 '1+A': 'I1_L-2C1R0',
 '2+AT': 'I2_L-2C2R1',
 '2+TC': 'I2_L-1C2R2',
 '2+GA': 'I2_L-3C2R0',
 '1+C': 'I1_L-1R0',
 '1+G': 'I1_L-1R0',
 '2+AA': 'I2_L-1R0',
 '2+AC': 'I2_L-1R0',
 '2+AG': 'I2_L-1R0',
 '2+CA': 'I2_L-1R0',
 '2+CC': 'I2_L-1R0',
 '2+CG': 'I2_L-1R0',
 '2+CT': 'I2_L-1R0',
 '2+GC': 'I2_L-1R0',
 '2+GG': 'I2_L-1R0',
 '2+GT': 'I2_L-1R0',
 '2+TA': 'I2_L-1R0',
 '2+TG': 'I2_L-1R0',
 '2+TT': 'I2_L-1R0'}

# Correlate Observed and Predicted Outcomes

In [30]:
# collect predicted data into dataframe
rows = []
indices = []
for t in TEST_FILES:
    test_f = file_mapping[t]
    for method in data[t].keys():
        for target_site in common_oligos[t]:
            indels = np.array(data[t][method][target_site]["indels"]) 
            predicted = data[t][method][target_site]["predicted"].astype(float) # Q
            observed = data[t][method][target_site]["actual"].astype(float)/sum(data[t][method][target_site]["actual"].astype(float)) # P
            indices.append((test_f, method, target_site))
            correlation = np.corrcoef(predicted, observed)[0,1]
            kl_divergence = entropy(observed, predicted)
            js = jensenshannon(observed, predicted)
            rows.append([correlation, kl_divergence, js])

indices = pd.MultiIndex.from_tuples(indices, names=["Dataset", "Method", "Target Site"])
df = pd.DataFrame(rows, index=indices, columns=["Pearson's Correlation", "KL Divergence", "Jensen Shannon"])
inf_oligos = df[~np.isfinite(df["KL Divergence"])].index.get_level_values(2)
df = df[~df.index.get_level_values(2).isin(inf_oligos)]
df.to_csv("/Users/colm/repos/x-crisp/data/processed/Performance/overall.tsv", sep="\t")
df.groupby(["Dataset", "Method"]).mean()

Pearson's Correlation  KL Divergence  \
Dataset                 Method                                              
FORECasT HAP1           FORECasT                  0.803754       1.042945   
                        Lindel                    0.679010       1.278830   
                        X-CRISP-KLD               0.783982       0.978153   
                        X-CRISP-MSE               0.786163       1.042323   
FORECasT TREX           FORECasT                  0.811772       0.994903   
                        Lindel                    0.523650       2.226950   
                        X-CRISP-KLD               0.574435       1.851151   
                        X-CRISP-MSE               0.561548       1.878977   
FORECasT WT             FORECasT                  0.805425       1.025114   
                        Lindel                    0.699450       1.226292   
                        X-CRISP-KLD               0.804397       0.976785   
                        X-CRISP-MSE               0.804087       1.042490   
inDelphi NHEJ-deficient FORECasT                  0.602306       1.709244   
                        Lindel                    0.544682       1.912158   
                        X-CRISP-KLD               0.614042       1.594207   
                        X-CRISP-MSE               0.618707       1.639123   
inDelphi USO2 WT        FORECasT                  0.601051       1.587994   
                        Lindel                    0.563154       1.879306   
                        X-CRISP-KLD               0.596485       1.650867   
                        X-CRISP-MSE               0.608086       1.690292   
inDelphi WT             FORECasT                  0.849631       0.759394   
                        Lindel                    0.781823       0.907598   
                        X-CRISP-KLD               0.850338       0.701599   
                        X-CRISP-MSE               0.847360       0.764315   

                                     Jensen Shannon  
Dataset                 Method                       
FORECasT HAP1           FORECasT           0.479519  
                        Lindel             0.493676  
                        X-CRISP-KLD        0.431578  
                        X-CRISP-MSE        0.461153  
FORECasT TREX           FORECasT           0.471118  
                        Lindel             0.582608  
                        X-CRISP-KLD        0.549994  
                        X-CRISP-MSE        0.570838  
FORECasT WT             FORECasT           0.476344  
                        Lindel             0.489311  
                        X-CRISP-KLD        0.432257  
                        X-CRISP-MSE        0.462659  
inDelphi NHEJ-deficient FORECasT           0.570631  
                        Lindel             0.572272  
                        X-CRISP-KLD        0.532719  
                        X-CRISP-MSE        0.553597  
inDelphi USO2 WT        FORECasT           0.571592  
                        Lindel             0.572285  
                        X-CRISP-KLD        0.547160  
                        X-CRISP-MSE        0.566943  
inDelphi WT             FORECasT           0.423292  
                        Lindel             0.424074  
                        X-CRISP-KLD        0.375930  
                        X-CRISP-MSE        0.410596

In [43]:
# collect predicted data into dataframe
rows = []
indices = []
for t in TEST_FILES:
    test_f = file_mapping[t]
    common_indels = {}
    for method in models:
        for target_site in common_oligos[t]:
            indels = np.array(data[t][method][target_site]["indels"])
            mh = np.array(data[t][method][target_site]["mh"])
            if (t not in BASELINE_TEST_FILES) and ("X-CRISP" in method):
                mh = np.array(list(mh) + ([False] * 21))

            if target_site in common_indels:
                common_indels[target_site] = np.intersect1d(common_indels[target_site], indels[mh])
            else:
                common_indels[target_site] = indels[mh]
    
    for method in models:
        for target_site in common_oligos[t]:
            indels = np.array(data[t][method][target_site]["indels"])
            # deletions = np.array([not is_insertion(x, method) for x in indels])
            # mh = np.array(data[t][method][target_site]["mh"])
            mh = np.isin(indels, common_indels[target_site])
            predicted = data[t][method][target_site]["predicted"][mh].astype(float) # Q
            observed = data[t][method][target_site]["actual"][mh].astype(float) # P

            predicted = predicted/sum(predicted)
            observed = observed/sum(observed)

            # then calculate
            indices.append((test_f, method, target_site))
            correlation = np.corrcoef(predicted, observed)[0,1]
            kl_divergence = entropy(observed, predicted)
            js = jensenshannon(observed, predicted)
            rows.append([correlation, kl_divergence, js])

indices = pd.MultiIndex.from_tuples(indices, names=["Dataset", "Method", "Target Site"])
df = pd.DataFrame(rows, index=indices, columns=["Pearson's Correlation", "KL Divergence", "Jensen Shannon"])
inf_oligos = df[~np.isfinite(df["KL Divergence"])].index.get_level_values(2)
df = df[~df.index.get_level_values(2).isin(inf_oligos)]
df.to_csv("/Users/colm/repos/x-crisp/data/processed/Performance/mh.tsv", sep="\t")
df.groupby(["Dataset", "Method"]).mean()

/var/folders/5x/n2td64g918lgvdh3qm2x3bhc0000gn/T/ipykernel_87817/833530732.py:29: RuntimeWarning: invalid value encountered in divide
  observed = observed/sum(observed)


Pearson's Correlation  KL Divergence  \
Dataset                 Method                                              
FORECasT HAP1           FORECasT                  0.798337       0.785164   
                        Lindel                    0.665348       1.068126   
                        X-CRISP-KLD               0.779425       0.769617   
                        X-CRISP-MSE               0.768409       0.799039   
FORECasT TREX           FORECasT                  0.804058       0.748055   
                        Lindel                    0.452306       2.011822   
                        X-CRISP-KLD               0.535421       1.590446   
                        X-CRISP-MSE               0.508682       1.610324   
FORECasT WT             FORECasT                  0.801748       0.766997   
                        Lindel                    0.715466       0.934339   
                        X-CRISP-KLD               0.808512       0.737457   
                        X-CRISP-MSE               0.808669       0.759984   
inDelphi NHEJ-deficient FORECasT                  0.628329       1.290186   
                        Lindel                    0.572912       1.486041   
                        X-CRISP-KLD               0.640733       1.230130   
                        X-CRISP-MSE               0.649775       1.212514   
inDelphi USO2 WT        FORECasT                  0.632111       1.422130   
                        Lindel                    0.572269       1.643059   
                        X-CRISP-KLD               0.629002       1.429828   
                        X-CRISP-MSE               0.620203       1.428236   
inDelphi WT             FORECasT                  0.849800       0.540216   
                        Lindel                    0.784071       0.665300   
                        X-CRISP-KLD               0.856160       0.498422   
                        X-CRISP-MSE               0.857091       0.499730   

                                     Jensen Shannon  
Dataset                 Method                       
FORECasT HAP1           FORECasT           0.418312  
                        Lindel             0.451472  
                        X-CRISP-KLD        0.387503  
                        X-CRISP-MSE        0.402448  
FORECasT TREX           FORECasT           0.409546  
                        Lindel             0.560011  
                        X-CRISP-KLD        0.520102  
                        X-CRISP-MSE        0.532651  
FORECasT WT             FORECasT           0.414526  
                        Lindel             0.437492  
                        X-CRISP-KLD        0.387365  
                        X-CRISP-MSE        0.400687  
inDelphi NHEJ-deficient FORECasT           0.503506  
                        Lindel             0.514752  
                        X-CRISP-KLD        0.478629  
                        X-CRISP-MSE        0.484000  
inDelphi USO2 WT        FORECasT           0.520871  
                        Lindel             0.534348  
                        X-CRISP-KLD        0.506319  
                        X-CRISP-MSE        0.515786  
inDelphi WT             FORECasT           0.352727  
                        Lindel             0.370202  
                        X-CRISP-KLD        0.324798  
                        X-CRISP-MSE        0.331948

In [ ]:
def is_insertion(indel, method):
    if method in ["1NN", "KLD", "Lindel", "inDelphi"]:
        return indel in common_insertions or indel == "3" or indel == "3+X"
    if method == "FORECasT":
        return indel[0] == "I"

# collect predicted data into dataframe
rows = []
indices = []
for t in TEST_FILES:
    test_f = file_mapping[t]
    common_indels = {}
    for method in models:
        for target_site in common_oligos[t]:
            indels = np.array(data[t][method][target_site]["indels"])
            deletions = np.array([not is_insertion(x, method) for x in indels])
            mhless = np.invert(np.array(data[t][method][target_site]["mh"]))
            if (t not in BASELINE_TEST_FILES) and ("X-CRISP" in method):
                mhless = np.array(list(mhless) + ([False] * 21))
            mhless_deletions = deletions & mhless
            if target_site in common_indels:
                common_indels[target_site] = np.intersect1d(common_indels[target_site], indels[mhless_deletions])
            else:
                common_indels[target_site] = indels[mhless_deletions]


    for method in models:
        if method == "inDelphi": continue
        for target_site in common_oligos[t]:
            indels = np.array(data[t][method][target_site]["indels"])
            mhless_deletions = np.isin(indels, common_indels[target_site])
            predicted = data[t][method][target_site]["predicted"][mhless_deletions].astype(float) # Q
            observed = data[t][method][target_site]["actual"][mhless_deletions].astype(float) # P

            predicted = predicted/sum(predicted)
            observed = observed/sum(observed)

            # then calculate
            indices.append((test_f, method, target_site))
            correlation = np.corrcoef(predicted, observed)[0,1]
            kl_divergence = entropy(observed, predicted)
            js = jensenshannon(observed, predicted)
            rows.append([correlation, kl_divergence, js])

indices = pd.MultiIndex.from_tuples(indices, names=["Dataset", "Method", "Target Site"])
df = pd.DataFrame(rows, index=indices, columns=["Pearson's Correlation", "KL Divergence", "Jensen Shannon"])
inf_oligos = df[~np.isfinite(df["KL Divergence"])].index.get_level_values(2)
df = df[~df.index.get_level_values(2).isin(inf_oligos)]
df.to_csv("/Users/colm/repos/x-crisp/data/processed/Performance/mhless.tsv", sep="\t")
df.groupby(["Dataset", "Method"]).mean()

In [ ]:
# copy takes about 14 seconds
ins_data = copy.deepcopy(data)

fasta_files = {
    "test": "../../src/data/FORECasT/test.fasta",
    "0105-mESC-Lib1-Cas9-Tol2-BioRep2-techrep1": "../../src/data/inDelphi/LibA.fasta"
}

guides = {}

for ff in fasta_files:
    guides.update(get_details_from_fasta(ff))

ins_mapping = {}
t = "test"
all_1bp_insertions = np.array(["1+A", "1+C", "1+G", "1+T"])
for t in TEST_FILES:
    ins_mapping[t] = {}
    for o in common_oligos[t]:
        g = guides[o]
        cutsite = g["PAM Index"] - 3
        ins_mapping[t][o] = {}
        FORECasT_insertions = [i for i in data[t]["FORECasT"][o]["indels"] if "I1" in i]
        FORECasT_rep_insertions = [i for i in FORECasT_insertions if "C" in i]
        FORECasT_norep_insertions = [i for i in FORECasT_insertions if "C" not in i]
        if len(FORECasT_norep_insertions) != 1: print("wtf")
        for i in FORECasT_rep_insertions:
            _, L, C, R = re.split("L|C|R", i)
            rep_nuc = g["TargetSequence"][cutsite + int(R) -1]
            ins_mapping[t][o][i] = "1+{}".format(rep_nuc)
        for i in FORECasT_norep_insertions:
            ins_mapping[t][o][i] = list(np.setdiff1d(all_1bp_insertions, list(ins_mapping[t][o].values()))) 

ins_mapping[t][o]

def is_forecast_insertion(indel):
        return indel[:2] == "I1"

# collect predicted data into dataframe
rows = []
indices = []
for t in TEST_FILES:
    test_f = file_mapping[t]
    common_indels = {}
    # for method in ["inDelphi"]:
    for method in [m for m in models if m != "FORECasT"]:
        for target_site in common_oligos[t]:
            new_predicted = []
            new_observed = []
            new_indels = []
            indels = np.array(data[t][method][target_site]["indels"])
            predicted = pd.Series(list(data[t][method][target_site]["predicted"]), index=indels)
            observed = pd.Series(list(data[t][method][target_site]["actual"]), index=indels)

            predicted = predicted/sum(predicted)
            observed = observed/sum(observed)

            for i in ins_mapping[t][target_site]:
                new_predicted.append(predicted.loc[ins_mapping[t][target_site][i]].sum())
                new_observed.append(observed.loc[ins_mapping[t][target_site][i]].sum())
                new_indels.append(i)

            ins_data[t][method][target_site]["indels"] = np.array(new_indels)
            ins_data[t][method][target_site]["predicted"] = np.array(new_predicted)
            ins_data[t][method][target_site]["actual"] = np.array(new_observed)

    for method in ins_data[t].keys():
        for target_site in common_oligos[t]:
            indels = ins_data[t][method][target_site]["indels"]
            insertions = np.array([is_forecast_insertion(x) for x in indels])
            predicted = ins_data[t][method][target_site]["predicted"][insertions].astype(float) # Q
            observed = ins_data[t][method][target_site]["actual"][insertions].astype(float) # P

            predicted = predicted/sum(predicted)
            observed = observed/sum(observed)

            # then calculate
            indices.append((test_f, method, target_site))
            correlation = np.corrcoef(predicted, observed)[0,1]
            kl_divergence = entropy(observed, predicted)
            js = jensenshannon(observed, predicted)

            rows.append([correlation, kl_divergence, js])

indices = pd.MultiIndex.from_tuples(indices, names=["Dataset", "Method", "Target Site"])
df = pd.DataFrame(rows, index=indices, columns=["Pearson's Correlation", "KL Divergence", "Jensen Shannon"])
df.to_csv("/Users/colm/repos/x-crisp/data/processed/Performance/insertion.tsv", sep="\t")
df.groupby(["Dataset", "Method"]).mean()

In [ ]:
from sklearn.metrics import precision_score, matthews_corrcoef, recall_score, f1_score


precisions = [.2, .3, .4, .5, .6, .7]
precision_X_mcc = {}

indices = []
rows_prec = []
rows_mcc = []
rows_recall = []
rows_f1_score = []
for t in TEST_FILES:
    test_f = file_mapping[t]
    for method in data[t].keys():
    # for method in ["inDelphi"]:
        row_prec = []
        row_mcc = []
        row_recall = []
        row_f1_score = []
        for p in precisions:
            precision_X_preds = []
            precision_X_labels = []
            for target_site in common_oligos[t]:
                pred = np.array(data[t][method][target_site]["predicted"])
                x = sum((pred/sum(pred)) > p) == 1
                precision_X_preds.append(x)
                obs = np.array(data[t][method][target_site]["actual"])
                y = sum((obs/sum(obs)) > p) == 1
                precision_X_labels.append(y)
            row_prec.append(precision_score(precision_X_labels, precision_X_preds))
            row_mcc.append(matthews_corrcoef(precision_X_labels, precision_X_preds))
            row_recall.append(recall_score(precision_X_labels, precision_X_preds))
            row_f1_score.append(recall_score(precision_X_labels, precision_X_preds))
        print("Finished {}".format(method))
        rows_prec.append(row_prec)
        rows_mcc.append(row_mcc) 
        rows_recall.append(row_recall)
        rows_f1_score.append(row_f1_score)    
        indices.append((test_f, method))
        
        
  
indices = pd.MultiIndex.from_tuples(indices, names=["Dataset", "Method"])
df_mcc = pd.DataFrame(rows_mcc, index=indices, columns=["Precision-{}".format(p) for p in precisions])
df_prec = pd.DataFrame(rows_prec, index=indices, columns=["Precision-{}".format(p) for p in precisions])
df_recall = pd.DataFrame(rows_recall, index=indices, columns=["Precision-{}".format(p) for p in precisions])
df_f1_score = pd.DataFrame(rows_f1_score, index=indices, columns=["Precision-{}".format(p) for p in precisions])

df_mcc.to_csv("/Users/colm/repos/x-crisp/data/processed/Performance/precision-X-mcc.tsv", sep="\t")
df_prec.to_csv("/Users/colm/repos/x-crisp/data/processed/Performance/precision-X-prec.tsv", sep="\t")
df_recall.to_csv("/Users/colm/repos/x-crisp/data/processed/Performance/precision-X-recall.tsv", sep="\t")
df_f1_score.to_csv("/Users/colm/repos/x-crisp/data/processed/Performance/precision-X-f1_score.tsv", sep="\t")

# Summary Statistics Comparisons

In [ ]:
from sklearn.metrics import mean_squared_error

# collect predicted data into dataframe
def is_1bp_deletion(indel, method):
    if method in ["1NN", "KLD", "Lindel", "FORECasT"]:
        return indel.split("+")[-1] == "1"
    if method == "inDelphi":
        return indel.split("+")[-1] == "1" or indel == "DL1"

def is_1bp_insertion(indel, method):
    if method in ["1NN", "KLD", "Lindel", "inDelphi"]:
        return indel in ['1+A', '1+C', '1+G', '1+T']
    if method == 'FORECasT':
        return indel[:2] == "I1"

def is_insertion(indel, method):
    if method in ["1NN", "KLD", "Lindel", "inDelphi"]:
        return indel in common_insertions or indel == "3" or indel == "3+X"
    if method == "FORECasT":
        return indel[0] == "I"

def is_frameshift(indel, method, t = "any"):
    length = None
    if method in ["1NN", "KLD", "Lindel"]:
        if is_insertion(indel, method):
            length = int(indel[0]) 
        else:
            length = int(indel.split("+")[-1]) 
    if method == "inDelphi":
        if is_insertion(indel, method):
            length = int(indel[0])
        elif indel[:2] == "DL":
            length = int(indel[2:])
        else:
            length = int(indel.split("+")[-1]) 
    if method == "FORECasT":
        if is_insertion(indel, method):
            length = int(indel.split("_")[0][1:])
        else:
            length = int(indel.split("+")[-1]) 
        
    allonebpframeshifts = np.array([1, 4, 7, 10, 13, 16, 19, 22, 25, 28])
    alltwobpframeshifts = allonebpframeshifts + 1

    if t == 1:
        return length in allonebpframeshifts
    elif t == 2:
        return length in alltwobpframeshifts
    else:
        return length % 3 != 0

rows = []
indices = []
for t in TEST_FILES:
    test_f = file_mapping[t]
    for method in data[t].keys():

        # deletion frequency
        # one basepair deletions
        preddelfreq = []
        obsdelfreq = []
        for target_site in common_oligos[t]:
            indels = np.array(data[t][method][target_site]["indels"]) 
            dels = [not is_insertion(x, method) for x in indels]
            preddelratio = sum(np.array(data[t][method][target_site]["predicted"])[dels])/sum(data[t][method][target_site]["predicted"])
            preddelfreq.append(preddelratio)
            obsdelratio = sum(np.array(data[t][method][target_site]["actual"])[dels])/sum(data[t][method][target_site]["actual"])
            obsdelfreq.append(obsdelratio)
        delmse = mean_squared_error(preddelfreq, obsdelfreq)


        # one basepair deletions
        predonebpdelfreq = []
        obsonebpdelfreq = []
        for target_site in common_oligos[t]:
            indels = np.array(data[t][method][target_site]["indels"]) 
            onebpdels = [is_1bp_deletion(x, method) for x in indels]
            predonebpdelratio = sum(np.array(data[t][method][target_site]["predicted"])[onebpdels])/sum(data[t][method][target_site]["predicted"])
            predonebpdelfreq.append(predonebpdelratio)
            obsonebpdelratio = sum(np.array(data[t][method][target_site]["actual"])[onebpdels])/sum(data[t][method][target_site]["actual"])
            obsonebpdelfreq.append(obsonebpdelratio)
        onebpdelmse = mean_squared_error(predonebpdelfreq, obsonebpdelfreq)


        # one basepair insertions
        predonebpinsfreq = []
        obsonebpinsfreq = []
        for target_site in common_oligos[t]:
            indels = np.array(data[t][method][target_site]["indels"]) 
            onebpins = [is_1bp_insertion(x, method) for x in indels]
            predonebpinsratio = sum(np.array(data[t][method][target_site]["predicted"])[onebpins])/sum(data[t][method][target_site]["predicted"])
            predonebpinsfreq.append(predonebpinsratio)
            obsonebpinsratio = sum(np.array(data[t][method][target_site]["actual"])[onebpins])/sum(data[t][method][target_site]["actual"])
            obsonebpinsfreq.append(obsonebpinsratio)
        onebpinsmse = mean_squared_error(predonebpinsfreq, obsonebpinsfreq)

        # one bp frameshift
        predonebpframeshiftfreq = []
        obsonebpframeshiftfreq = []
        for target_site in common_oligos[t]:
            indels = np.array(data[t][method][target_site]["indels"]) 
            onebpframeshift = [is_frameshift(x, method, 1) for x in indels]
            predonebpframeshiftratio = sum(np.array(data[t][method][target_site]["predicted"])[onebpframeshift])/sum(data[t][method][target_site]["predicted"])
            predonebpframeshiftfreq.append(predonebpframeshiftratio)
            obsonebpframeshiftratio = sum(np.array(data[t][method][target_site]["actual"])[onebpframeshift])/sum(data[t][method][target_site]["actual"])
            obsonebpframeshiftfreq.append(obsonebpframeshiftratio)
        onebpframeshiftmse = mean_squared_error(predonebpframeshiftfreq, obsonebpframeshiftfreq)

        # two bp frameshift
        predtwobpframeshiftfreq = []
        obstwobpframeshiftfreq = []
        for target_site in common_oligos[t]:
            indels = np.array(data[t][method][target_site]["indels"]) 
            twobpframeshift = [is_frameshift(x, method, 2) for x in indels]
            predtwobpframeshiftratio = sum(np.array(data[t][method][target_site]["predicted"])[twobpframeshift])/sum(data[t][method][target_site]["predicted"])
            predtwobpframeshiftfreq.append(predtwobpframeshiftratio)
            obstwobpframeshiftratio = sum(np.array(data[t][method][target_site]["actual"])[twobpframeshift])/sum(data[t][method][target_site]["actual"])
            obstwobpframeshiftfreq.append(obstwobpframeshiftratio)
        twobpframeshiftmse = mean_squared_error(predtwobpframeshiftfreq, obstwobpframeshiftfreq)
        
        # frameshift
        predframeshiftfreq = []
        obsframeshiftfreq = []
        for target_site in common_oligos[t]:
            indels = np.array(data[t][method][target_site]["indels"]) 
            frameshift = [is_frameshift(x, method, "any") for x in indels]
            predframeshiftratio = sum(np.array(data[t][method][target_site]["predicted"])[frameshift])/sum(data[t][method][target_site]["predicted"])
            predframeshiftfreq.append(predframeshiftratio)
            obsframeshiftratio = sum(np.array(data[t][method][target_site]["actual"])[frameshift])/sum(data[t][method][target_site]["actual"])
            obsframeshiftfreq.append(obsframeshiftratio)
        frameshiftmse = mean_squared_error(predframeshiftfreq, obsframeshiftfreq)


        rows.append([delmse, onebpdelmse, onebpinsmse, onebpframeshiftmse, twobpframeshiftmse, frameshiftmse])
        indices.append((test_f, method))
  
indices = pd.MultiIndex.from_tuples(indices, names=["Dataset", "Method"])
df = pd.DataFrame(rows, index=indices, columns=["Deletion", "1BP Deletion", "1BP Insertion", "1BP Frameshift", "2BP Frameshift", "Frameshift"])
df.groupby(["Dataset", "Method"])

### CROTON comparisons

In [ ]:
CROTON_F = "/Users/colm/repos/output/local/model_predictions/CROTON/{}_new.pkl"

summ_data = {}

for i, t in enumerate(TEST_FILES):
    croton_d = pkl.load(open(CROTON_F.format(t), 'rb'))
    summ_data[t] = croton_d

croton_i = pd.MultiIndex.from_product([[file_mapping[t] for t in TEST_FILES], ["CROTON"]], names=["Dataset", "Method"])


croton_del_freq_mse = []
croton_prob_1bpins_mse = []
croton_prob_1bpdel_mse = []
croton_one_bp_frameshift_mse = []
croton_two_bp_frameshift_mse = []
croton_frameshift_mse = []

for t in TEST_FILES:
    croton_del_freq_mse.append(mean_squared_error(summ_data[t]["predicted"]["del_freq"], summ_data[t]["actual"]["del_freq"]))
    croton_prob_1bpins_mse.append(mean_squared_error(summ_data[t]["predicted"]["prob_1bpins"], summ_data[t]["actual"]["prob_1bpins"]))
    croton_prob_1bpdel_mse.append(mean_squared_error(summ_data[t]["predicted"]["prob_1bpdel"], summ_data[t]["actual"]["prob_1bpdel"]))
    croton_one_bp_frameshift_mse.append(mean_squared_error(summ_data[t]["predicted"]["one_bp_frameshift"], summ_data[t]["actual"]["one_bp_frameshift"]))
    croton_two_bp_frameshift_mse.append(mean_squared_error(summ_data[t]["predicted"]["two_bp_frameshift"], summ_data[t]["actual"]["two_bp_frameshift"]))
    croton_frameshift_mse.append(mean_squared_error(summ_data[t]["predicted"]["frameshift"], summ_data[t]["actual"]["frameshift"]))

croton_mse_d = pd.DataFrame({
    "Deletion": croton_del_freq_mse,
    "1BP Insertion": croton_prob_1bpins_mse,
    "1BP Deletion": croton_prob_1bpdel_mse,
    "1BP Frameshift": croton_one_bp_frameshift_mse,
    "2BP Frameshift": croton_two_bp_frameshift_mse,
    "Frameshift": croton_frameshift_mse,
}, index=croton_i)

croton_mse_d

In [ ]:
df = pd.concat([df, croton_mse_d]).sort_index()
df.to_csv("/Users/colm/repos/x-crisp/data/processed/Performance/stats_comparison.tsv", sep="\t")
df